In [ ]:
# ruff: noqa: E402
import math
import warnings
from typing import Dict, Literal
import time

warnings.simplefilter("ignore")
import delu  # Deep Learning Utilities: https://github.com/Yura52/delu
import numpy as np
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm
import json
import sys

warnings.resetwarnings()

from rtdl_revisiting_models import MLP, ResNet, FTTransformer

from fourierDistill import *
from featurizer import BinaryTransformer

In [ ]:
n_inter = 3
k_cv = 3

In [ ]:
#dataset = sklearn.datasets.fetch_california_housing(as_frame = True)
dataset = sklearn.datasets.load_iris(as_frame = True)
X: np.ndarray = dataset["data"]
Y: np.ndarray = dataset["target"]

all_idx = np.arange(len(Y))
train_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8, random_state = 0
)
# train_idx, val_idx = sklearn.model_selection.train_test_split(
#     trainval_idx, train_size=0.8, random_state = 0
# )
y_train = Y.loc[train_idx]
y_test = Y.loc[test_idx]

In [ ]:
bt_bin3 = BinaryTransformer(depth = 3, bit = False)
X_train_bin3 = bt_bin3.fit_and_transform(X.loc[train_idx, :], Y.loc[train_idx])
X_test_bin3 = bt_bin3.transform(X.loc[test_idx, :])

In [ ]:
X_train_bin3.shape

In [ ]:
ftd_bin3 = FTDistillClassifierCV(
                 pre_interaction='l1l2', 
                 pre_lam1=0.01, 
                 pre_lam2=0.01,
                 pre_max_features=None,
                 post_interaction='l1l2', 
                 post_lam1=0.1, 
                 post_lam2=0.1,
                 post_max_features=None,
                 size_interactions=2)


start = time.time()
ftd_bin3.fit(X_train_bin3, y_train, bt_bin3.no_interaction)
end = time.time()

In [ ]:
ftd_bin3.post_interaction_model.C

In [ ]:
ftd_bin3.post_interaction_model.C

In [ ]:
np.mean(ftd_bin3.predict(X_test_bin3)== y_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(ftd_bin3.predict(X_train_bin3), y_train)

In [ ]:
ftd_bin3.post_lam2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
model_list = [ftd_bin3, ftd_bit3, ftd_bit4]
model_names = ['(bin3, true, train)', '(bit3, true, train)', '(bit4, true, train)']

r2_df = pd.DataFrame()
r2_df['Model'] = model_names

In [ ]:
r2_df['Train R2'] = [r2_score(ftd_bin3.predict(X_train_bin3), y_train),r2_score(ftd_bit3.predict(X_train_bit3), y_train), r2_score(ftd_bit4.predict(X_train_bit4), y_train)]
r2_df['Test R2'] = [r2_score(ftd_bin3.predict(X_test_bin3), y_test),r2_score(ftd_bit3.predict(X_test_bit3), y_test), r2_score(ftd_bit4.predict(X_test_bit4), y_test)]

In [ ]:
r2_df['Train Time'] = train_time
r2_df['Total Num Features'] = [len(m.regression_model.coef_) for m in model_list]
r2_df['Num Selected Features'] = [sum(m.regression_model.coef_ != 0) for m in model_list]

In [ ]:
r2_df

In [ ]:
r2_df.to_csv('r2/binarize_prediction.csv')

In [ ]:
[m.regression_model.reg_param for m in model_list]

In [ ]:
ftd_bit4.regression_model.alpha_